# SymPy amplitude model

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

## Generate transitions

In [ ]:
import expertsystem as es

result = es.reaction.generate(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)(980)", "f(0)(1500)"],
    allowed_interaction_types=["strong", "EM"],
)

In [ ]:
import graphviz

graphs = result.collapse_graphs()
dot = es.io.convert_to_dot(graphs)
graphviz.Source(dot)

## Build SymPy expression

In [ ]:
model = es.amplitude.generate_sympy(result)
expression = model.expression
expression.top

In [ ]:
expression.top

In [ ]:
expression.top.subs(expression.intensities)

In [ ]:
import sympy as sy

initial_values = {k: v.value for k, v in model.parameters.items()}
evaluated_wigner_d = sy.expand(
    expression.full_expression.subs(initial_values).doit()
)
evaluated_wigner_d

## Visualize

In this case ($J/\psi \to \gamma f_0, f_0 \to \pi^0\pi^0$) _without dynamics_, the total intensity is only dependent on the $\theta$ angle between the two $\pi^0$'s:

In [ ]:
import sympy as sy

theta = next(iter(evaluated_wigner_d.free_symbols))
sy.plot(
    evaluated_wigner_d,
    (theta, 0, sy.pi),
    axis_center=(0, 0),
    ylabel="$I$",
    ylim=(0, 8),
);

This intensity is built up of four components:

In [ ]:
from IPython.display import display

for intensity in expression.intensities:
    display(intensity)

In [ ]:
import sympy as sy

theta = next(iter(evaluated_wigner_d.free_symbols))

plots = list()
colors = ["red", "blue", "green", "purple"]

total = 0
for i, (label, intensity) in enumerate(expression.intensities.items()):
    total += (
        intensity.subs(expression.amplitudes)
        .subs(expression.dynamics)
        .subs(initial_values)
        .doit()
    )
    plots.append(
        sy.plot(
            total,
            (theta, 0, sy.pi),
            axis_center=(0, 0),
            ylabel="$I$",
            ylim=(0, 8),
            line_color=colors[i],
            show=False,
            label=f"${label.name}$",
            legend=True,
        )
    )
for i in range(1, 4):
    plots[0].extend(plots[i])
plots[0].show()

## Set dynamics

In [ ]:
expression.dynamics

In [ ]:
from expertsystem.amplitude.dynamics import relativistic_breit_wigner

m = sy.Symbol("m(\pi^0\pi^0)", real=True)
resonances = [
    model.particles["f(0)(980)"],
    model.particles["f(0)(1500)"],
]
for p in resonances:
    dynamics_label = next(
        iter(
            filter(
                lambda d: d.name.startswith(f"D[{p.latex}"),
                expression.dynamics,
            )
        )
    )
    dynamics_expression = relativistic_breit_wigner(
        mass=m,
        mass0=p.mass,
        gamma0=p.width,
    )
    expression.dynamics[dynamics_label] = dynamics_expression

assert len(expression.dynamics) == 4

In [ ]:
full_expression = expression.full_expression.subs(initial_values)
full_expression.free_symbols

In [ ]:
angle = 0
angle_substitutions = {
    s: angle for s in full_expression.free_symbols if s != m
}
intensity = full_expression.doit().subs(angle_substitutions)

In [ ]:
# integration_variables = [s for s in full_expression.free_symbols if s != mass_symbol]
# sy.integrate(
#     sy.re(full_expression.doit()),
#     *(
#         (x, -sy.pi, sy.pi) if "phi" in x.name else (x, 0, sy.pi)
#         for x in integration_variables
#     ),
# )

In [ ]:
sy.plot(
    intensity,
    (m, 0, 3),
    axis_center=(0, 0),
    xlabel=f"${m.name}$",
    ylabel="$I$",
);